# Homework 06: JSON

## Nicolás Echeverri Rojas


- The Python dict string uses single quotation marks,  and JSON enforces double quotation marks. You can nest tuple in Python dict.


- Para acceder a los daton de JSON: data['key_name'] o data.get('key_name') (en este si la key no existe retorna None)

- En inspirehep.net escribir /api/ después de inspirehep para tener el archivo de JSON de la página web

- Están organizados por diccionarios y listas, para las listas se puede acceder a los elementos de manera indexada normal

## 1. 

Descargue el JSON con la lista de paises del siguiente link: https://datahub.io/core/country-list/r/data.json
y escoja aleatoriamente un país de Europa  (Ejemplo que es abajo usa Colombia pero con pais de europa es mas facil)

Use los diferentes API endpoints de inspire-hep https://inspirehep.net/ Para extraer la lista de investigadores de una institución de ese país  en esa base de datos

In [169]:
import requests
import pandas as pd

In [26]:
institution_link=requests.get('https://datahub.io/core/country-list/r/data.json')

In [29]:
institution_link.json()

[{'Code': 'AF', 'Name': 'Afghanistan'},
 {'Code': 'AX', 'Name': 'Åland Islands'},
 {'Code': 'AL', 'Name': 'Albania'},
 {'Code': 'DZ', 'Name': 'Algeria'},
 {'Code': 'AS', 'Name': 'American Samoa'},
 {'Code': 'AD', 'Name': 'Andorra'},
 {'Code': 'AO', 'Name': 'Angola'},
 {'Code': 'AI', 'Name': 'Anguilla'},
 {'Code': 'AQ', 'Name': 'Antarctica'},
 {'Code': 'AG', 'Name': 'Antigua and Barbuda'},
 {'Code': 'AR', 'Name': 'Argentina'},
 {'Code': 'AM', 'Name': 'Armenia'},
 {'Code': 'AW', 'Name': 'Aruba'},
 {'Code': 'AU', 'Name': 'Australia'},
 {'Code': 'AT', 'Name': 'Austria'},
 {'Code': 'AZ', 'Name': 'Azerbaijan'},
 {'Code': 'BS', 'Name': 'Bahamas'},
 {'Code': 'BH', 'Name': 'Bahrain'},
 {'Code': 'BD', 'Name': 'Bangladesh'},
 {'Code': 'BB', 'Name': 'Barbados'},
 {'Code': 'BY', 'Name': 'Belarus'},
 {'Code': 'BE', 'Name': 'Belgium'},
 {'Code': 'BZ', 'Name': 'Belize'},
 {'Code': 'BJ', 'Name': 'Benin'},
 {'Code': 'BM', 'Name': 'Bermuda'},
 {'Code': 'BT', 'Name': 'Bhutan'},
 {'Code': 'BO', 'Name': 'Bo

### Queremos analizar las instituciones de Alemania

In [36]:
for i,dic in enumerate(institution_link.json()):
    if 'Germany' in dic.get('Name'):
        print(i)

82


In [40]:
name_country = institution_link.json()[82].get('Name').lower()
name_country

'germany'

### A)
Use el API de institutions para extraer la lista de instituciones del país, por ejemplo: colombia (mejor si es Europa, por completo de datos )
https://inspirehep.net/api/institutions?q=colombia 

In [42]:
country_link =requests.get('https://inspirehep.net/api/institutions?q=%s'%name_country)

### Implementación total (ver desarrollo paso a paso para discusión más detallada)

In [164]:
def extract_authors(name_country,  index_institution):
    
    #A) Obteniendo el JSON de las instituciones 
    country_link =requests.get('https://inspirehep.net/api/institutions?q=%s'%name_country)
    
    
    
    #B) Obteniendo el número de papers y legacy icn de la institución con un índice dado
    number_papers = (country_link.json().get('hits').get('hits')
                     [ index_institution].get('metadata').get('number_of_papers'))
    
    legacy_ICN = (country_link.json().get('hits').get('hits')
                  [index_institution].get('metadata').get('legacy_ICN'))

    print(f"number of papers: {number_papers}, legacy_ICN: {legacy_ICN}")
    
    
    
    
    #C) Accediendo a los artículos de menos de 10 autores
    legacy_ICN_link = legacy_ICN.replace(' ','+')
    articles = requests.get(f'https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{legacy_ICN_link }+and+ac+1->+10')
    
    #D) Accediendo a los autores 
    
    n_authors = len(articles.json().get('hits').get('hits')[index_institution].get('metadata').get('authors')) 
    information_list =[]
    
    for index_author in range(n_authors):
        author_link = (articles.json().get('hits').get('hits')[index_institution]
                       .get('metadata').get('authors')[index_author].get('record').get('$ref'))

        author_web = requests.get(author_link)

        institution_name = (articles.json().get('hits').get('hits')
                            [index_institution].get('metadata').get('authors')[index_author]
                            .get('raw_affiliations')[0]['value'])

        author_name = author_web.json().get('metadata').get('name')['value']
        #print(institution_name)
        #print(author_web.json().get('metadata'))
        information_dict = {'author_name':author_name, 'institution_name':institution_name}
        information_list.append(information_dict)
    
    return  information_list

In [166]:
information_list = extract_authors(name_country= 'germany',  index_institution = 0)

number of papers: 20, legacy_ICN: Inst. Photonic Tech., Jena


### Creación de tabla con la información del dataframe

In [170]:
pd.DataFrame(information_list)

,author_name,institution_name
0,"Ponce, Meritxell Cabrejo","Friedrich Schiller University, Max-Wien-Platz ..."
1,"Muniz, André Luiz Marques",Fraunhofer Institute for Applied Optics and Pr...
2,"Ancsin, Philippe",Fraunhofer Institute for Applied Optics and Pr...
3,"Spiess, Christopher","Friedrich Schiller University, Max-Wien-Platz ..."
4,"Steinlechner, Fabian",Fraunhofer Institute for Applied Optics and Pr...


### Solo se pudo obtener el nombre del autor y la institución donde labora ya que no se encuentra más información el los metadatos de la páguna web de cada autor author_web.json().get('metadata')

## Desarrollos paso a paso 

In [48]:
country_link.json()

{'hits': {'hits': [{'updated': '2021-06-25T22:43:46.256816+00:00',
    'metadata': {'number_of_papers': 20,
     'addresses': [{'cities': ['Jena'],
       'postal_code': '07745',
       'country_code': 'DE',
       'postal_address': ['Institute of Photonic Technology (IPHT)',
        'Albert Einstein Str. 9',
        'P.O. Box 100239',
        'D-07745 Jena'],
       'country': 'Germany'}],
     'self': {'$ref': 'https://inspirehep.net/api/institutions/911171'},
     'urls': [{'value': 'http://www.ipht-jena.de/'}],
     '$schema': 'https://inspirehep.net/schemas/records/institutions.json',
     'legacy_ICN': 'Inst. Photonic Tech., Jena',
     'control_number': 911171,
     'legacy_version': '20150317115351.0',
     'legacy_creation_date': '2007-11-14',
     'institution_hierarchy': [{'name': 'GERMANY'}],
     'external_system_identifiers': [{'value': 'INST-100730',
       'schema': 'SPIRES'}]},
    'created': '2007-11-14T00:00:00+00:00',
    'links': {'json': 'https://inspirehep.net/ap

In [47]:
country_link.json().keys()

dict_keys(['hits', 'links'])

In [52]:
country_link.json().get('hits')

{'hits': [{'updated': '2021-06-25T22:43:46.256816+00:00',
   'metadata': {'number_of_papers': 20,
    'addresses': [{'cities': ['Jena'],
      'postal_code': '07745',
      'country_code': 'DE',
      'postal_address': ['Institute of Photonic Technology (IPHT)',
       'Albert Einstein Str. 9',
       'P.O. Box 100239',
       'D-07745 Jena'],
      'country': 'Germany'}],
    'self': {'$ref': 'https://inspirehep.net/api/institutions/911171'},
    'urls': [{'value': 'http://www.ipht-jena.de/'}],
    '$schema': 'https://inspirehep.net/schemas/records/institutions.json',
    'legacy_ICN': 'Inst. Photonic Tech., Jena',
    'control_number': 911171,
    'legacy_version': '20150317115351.0',
    'legacy_creation_date': '2007-11-14',
    'institution_hierarchy': [{'name': 'GERMANY'}],
    'external_system_identifiers': [{'value': 'INST-100730',
      'schema': 'SPIRES'}]},
   'created': '2007-11-14T00:00:00+00:00',
   'links': {'json': 'https://inspirehep.net/api/institutions/911171?format=j

In [49]:
country_link.json().get('hits').keys()

dict_keys(['hits', 'total'])

In [46]:
country_link.json().get('hits').get('hits')

[{'updated': '2021-06-25T22:43:46.256816+00:00',
  'metadata': {'number_of_papers': 20,
   'addresses': [{'cities': ['Jena'],
     'postal_code': '07745',
     'country_code': 'DE',
     'postal_address': ['Institute of Photonic Technology (IPHT)',
      'Albert Einstein Str. 9',
      'P.O. Box 100239',
      'D-07745 Jena'],
     'country': 'Germany'}],
   'self': {'$ref': 'https://inspirehep.net/api/institutions/911171'},
   'urls': [{'value': 'http://www.ipht-jena.de/'}],
   '$schema': 'https://inspirehep.net/schemas/records/institutions.json',
   'legacy_ICN': 'Inst. Photonic Tech., Jena',
   'control_number': 911171,
   'legacy_version': '20150317115351.0',
   'legacy_creation_date': '2007-11-14',
   'institution_hierarchy': [{'name': 'GERMANY'}],
   'external_system_identifiers': [{'value': 'INST-100730',
     'schema': 'SPIRES'}]},
  'created': '2007-11-14T00:00:00+00:00',
  'links': {'json': 'https://inspirehep.net/api/institutions/911171?format=json'},
  'id': '911171'},
 {'u

In [51]:
country_link.json().get('hits').get('hits')

[{'updated': '2021-06-25T22:43:46.256816+00:00',
  'metadata': {'number_of_papers': 20,
   'addresses': [{'cities': ['Jena'],
     'postal_code': '07745',
     'country_code': 'DE',
     'postal_address': ['Institute of Photonic Technology (IPHT)',
      'Albert Einstein Str. 9',
      'P.O. Box 100239',
      'D-07745 Jena'],
     'country': 'Germany'}],
   'self': {'$ref': 'https://inspirehep.net/api/institutions/911171'},
   'urls': [{'value': 'http://www.ipht-jena.de/'}],
   '$schema': 'https://inspirehep.net/schemas/records/institutions.json',
   'legacy_ICN': 'Inst. Photonic Tech., Jena',
   'control_number': 911171,
   'legacy_version': '20150317115351.0',
   'legacy_creation_date': '2007-11-14',
   'institution_hierarchy': [{'name': 'GERMANY'}],
   'external_system_identifiers': [{'value': 'INST-100730',
     'schema': 'SPIRES'}]},
  'created': '2007-11-14T00:00:00+00:00',
  'links': {'json': 'https://inspirehep.net/api/institutions/911171?format=json'},
  'id': '911171'},
 {'u

In [116]:
country_link.json().get('hits').get('hits')[0]

{'updated': '2021-06-25T22:43:46.256816+00:00',
 'metadata': {'number_of_papers': 20,
  'addresses': [{'cities': ['Jena'],
    'postal_code': '07745',
    'country_code': 'DE',
    'postal_address': ['Institute of Photonic Technology (IPHT)',
     'Albert Einstein Str. 9',
     'P.O. Box 100239',
     'D-07745 Jena'],
    'country': 'Germany'}],
  'self': {'$ref': 'https://inspirehep.net/api/institutions/911171'},
  'urls': [{'value': 'http://www.ipht-jena.de/'}],
  '$schema': 'https://inspirehep.net/schemas/records/institutions.json',
  'legacy_ICN': 'Inst. Photonic Tech., Jena',
  'control_number': 911171,
  'legacy_version': '20150317115351.0',
  'legacy_creation_date': '2007-11-14',
  'institution_hierarchy': [{'name': 'GERMANY'}],
  'external_system_identifiers': [{'value': 'INST-100730',
    'schema': 'SPIRES'}]},
 'created': '2007-11-14T00:00:00+00:00',
 'links': {'json': 'https://inspirehep.net/api/institutions/911171?format=json'},
 'id': '911171'}

In [56]:
country_link.json().get('hits').get('hits')[0].keys()

dict_keys(['updated', 'metadata', 'created', 'links', 'id'])

In [126]:
country_link.json().get('hits').get('hits')[0].get('metadata')

{'number_of_papers': 20,
 'addresses': [{'cities': ['Jena'],
   'postal_code': '07745',
   'country_code': 'DE',
   'postal_address': ['Institute of Photonic Technology (IPHT)',
    'Albert Einstein Str. 9',
    'P.O. Box 100239',
    'D-07745 Jena'],
   'country': 'Germany'}],
 'self': {'$ref': 'https://inspirehep.net/api/institutions/911171'},
 'urls': [{'value': 'http://www.ipht-jena.de/'}],
 '$schema': 'https://inspirehep.net/schemas/records/institutions.json',
 'legacy_ICN': 'Inst. Photonic Tech., Jena',
 'control_number': 911171,
 'legacy_version': '20150317115351.0',
 'legacy_creation_date': '2007-11-14',
 'institution_hierarchy': [{'name': 'GERMANY'}],
 'external_system_identifiers': [{'value': 'INST-100730', 'schema': 'SPIRES'}]}

In [105]:
number_papers, legacy_ICN = (country_link.json().get('hits').get('hits')[0].get('metadata').get('number_of_papers'),
country_link.json().get('hits').get('hits')[0].get('metadata').get('legacy_ICN') )

print(f"number of papers: {number_papers}, legacy_ICN: {legacy_ICN}")

number of papers: 20, legacy_ICN: Inst. Photonic Tech., Jena


### B)
Con el API de literatura obtenga el JSON con los artículos de menos de 10 autores usando el "legacy_ICN" de la siguiente manera

https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+Colombia,+U.+Natl.+and+ac+1->+10

aff: usa el valor de legacy_ICN
and: es un operador lógico
ac: establece los autores entre 1 y 10


In [106]:
legacy_ICN_link = legacy_ICN.replace(' ','+')
articles = requests.get(f'https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{legacy_ICN_link }+and+ac+1->+10')

In [107]:
articles.json().get('hits').get('hits')

[{'metadata': {'citation_count': 1,
   'authors': [{'raw_affiliations': [{'value': 'Friedrich Schiller University, Max-Wien-Platz 1, 07743 Jena, Germany'},
      {'value': 'Fraunhofer Institute for Applied Optics and Precision Engineering, Albert-Einstein-Str. 7, 07745 Jena, Germany'}],
     'full_name_unicode_normalized': 'ponce, meritxell cabrejo',
     'last_name': 'Ponce',
     'affiliations': [{'curated_relation': True,
       'record': {'$ref': 'https://inspirehep.net/api/institutions/909821'},
       'value': 'Jena U., TPI'},
      {'curated_relation': True,
       'record': {'$ref': 'https://inspirehep.net/api/institutions/910325'},
       'value': 'Fraunhofer Inst., Jena'}],
     'bai': 'M.C.Ponce.1',
     'uuid': '0d192fe9-e7dd-49ae-8b3c-4378cbdb4a4a',
     'full_name': 'Ponce, Meritxell Cabrejo',
     'record': {'$ref': 'https://inspirehep.net/api/authors/2015544'},
     'ids': [{'schema': 'ORCID', 'value': '0000-0003-0451-3834'},
      {'schema': 'INSPIRE BAI', 'value': 'M.

In [108]:
articles.json().get('hits').get('hits')[0].get('metadata')

{'citation_count': 1,
 'authors': [{'raw_affiliations': [{'value': 'Friedrich Schiller University, Max-Wien-Platz 1, 07743 Jena, Germany'},
    {'value': 'Fraunhofer Institute for Applied Optics and Precision Engineering, Albert-Einstein-Str. 7, 07745 Jena, Germany'}],
   'full_name_unicode_normalized': 'ponce, meritxell cabrejo',
   'last_name': 'Ponce',
   'affiliations': [{'curated_relation': True,
     'record': {'$ref': 'https://inspirehep.net/api/institutions/909821'},
     'value': 'Jena U., TPI'},
    {'curated_relation': True,
     'record': {'$ref': 'https://inspirehep.net/api/institutions/910325'},
     'value': 'Fraunhofer Inst., Jena'}],
   'bai': 'M.C.Ponce.1',
   'uuid': '0d192fe9-e7dd-49ae-8b3c-4378cbdb4a4a',
   'full_name': 'Ponce, Meritxell Cabrejo',
   'record': {'$ref': 'https://inspirehep.net/api/authors/2015544'},
   'ids': [{'schema': 'ORCID', 'value': '0000-0003-0451-3834'},
    {'schema': 'INSPIRE BAI', 'value': 'M.C.Ponce.1'}],
   'signature_block': 'PANCm',
 

## C)

Para al menos un artículo de esa institución, extraiga el URL del perfil de cada autor de esa institución que se encuentra dentro del campo "authors" en "record" y luego en "$ref". Por ejemplo

In [109]:
articles.json().get('hits').get('hits')[0].get('metadata').keys()

dict_keys(['citation_count', 'authors', 'publication_info', 'documents', 'citation_count_without_self_citations', 'citeable', 'references', 'keywords', '$schema', 'number_of_pages', 'preprint_date', 'author_count', 'public_notes', 'earliest_date', 'refereed', 'facet_author_name', 'core', 'license', 'journal_title_variants', 'arxiv_eprints', 'referenced_authors_bais', 'inspire_categories', 'figures', 'first_author', 'control_number', 'dois', 'document_type', 'texkeys', 'abstracts', 'primary_arxiv_category', 'titles', 'imprints', 'curated'])

In [110]:
articles.json().get('hits').get('hits')[0].get('metadata').get('authors')

[{'raw_affiliations': [{'value': 'Friedrich Schiller University, Max-Wien-Platz 1, 07743 Jena, Germany'},
   {'value': 'Fraunhofer Institute for Applied Optics and Precision Engineering, Albert-Einstein-Str. 7, 07745 Jena, Germany'}],
  'full_name_unicode_normalized': 'ponce, meritxell cabrejo',
  'last_name': 'Ponce',
  'affiliations': [{'curated_relation': True,
    'record': {'$ref': 'https://inspirehep.net/api/institutions/909821'},
    'value': 'Jena U., TPI'},
   {'curated_relation': True,
    'record': {'$ref': 'https://inspirehep.net/api/institutions/910325'},
    'value': 'Fraunhofer Inst., Jena'}],
  'bai': 'M.C.Ponce.1',
  'uuid': '0d192fe9-e7dd-49ae-8b3c-4378cbdb4a4a',
  'full_name': 'Ponce, Meritxell Cabrejo',
  'record': {'$ref': 'https://inspirehep.net/api/authors/2015544'},
  'ids': [{'schema': 'ORCID', 'value': '0000-0003-0451-3834'},
   {'schema': 'INSPIRE BAI', 'value': 'M.C.Ponce.1'}],
  'signature_block': 'PANCm',
  'first_name': 'Meritxell Cabrejo',
  'recid': 201

In [79]:
len(articles.json().get('hits').get('hits')[0].get('metadata').get('authors'))

5

In [162]:
articles.json().get('hits').get('hits')[0].get('metadata').get('authors')[4]

{'raw_affiliations': [{'value': 'Fraunhofer Institute for Applied Optics and Precision Engineering, Albert-Einstein-Str. 7, 07745 Jena, Germany'},
  {'value': 'Abbe Center of Photonics, Friedrich Schiller University Jena, Albert-Einstein-Str. 6, 07745 Jena, Germany'}],
 'full_name_unicode_normalized': 'steinlechner, fabian',
 'last_name': 'Steinlechner',
 'affiliations': [{'curated_relation': True,
   'record': {'$ref': 'https://inspirehep.net/api/institutions/908513'},
   'value': 'Fraunhofer Inst., Jena'},
  {'curated_relation': True,
   'record': {'$ref': 'https://inspirehep.net/api/institutions/911171'},
   'value': 'Inst. Photonic Tech., Jena'}],
 'bai': 'F.Steinlechner.1',
 'uuid': '4859f4a1-a8fb-4187-937c-c8fca63c0f08',
 'full_name': 'Steinlechner, Fabian',
 'record': {'$ref': 'https://inspirehep.net/api/authors/1961572'},
 'ids': [{'schema': 'ORCID', 'value': '0000-0003-0122-1182'},
  {'schema': 'INSPIRE BAI', 'value': 'F.Steinlechner.1'}],
 'signature_block': 'STANLACHNARf',
 

In [163]:
articles.json().get('hits').get('hits')[0].get('metadata').get('authors')[0].get('raw_affiliations')[0]['value']

'Friedrich Schiller University, Max-Wien-Platz 1, 07743 Jena, Germany'

In [91]:
articles.json().get('hits').get('hits')[0].get('metadata').get('authors')[1].keys()

dict_keys(['raw_affiliations', 'full_name_unicode_normalized', 'full_name', 'record', 'last_name', 'affiliations', 'ids', 'bai', 'signature_block', 'uuid', 'first_name', 'recid'])

In [92]:
author_link = articles.json().get('hits').get('hits')[0].get('metadata').get('authors')[1].get('record').get('$ref')
author_link 

'https://inspirehep.net/api/authors/2015545'

In [93]:
author_web = requests.get(author_link)

In [94]:
author_web.json()

{'uuid': '2aea382d-1672-4802-ac80-8f5ac2f1473a',
 'metadata': {'ids': [{'value': 'A.L.M.Muniz.1', 'schema': 'INSPIRE BAI'}],
  'name': {'value': 'Muniz, André Luiz Marques'},
  'stub': True,
  '$schema': 'https://inspirehep.net/schemas/records/authors.json',
  'control_number': 2015545},
 'id': '2015545',
 'revision_id': 0,
 'created': '2022-01-24T02:48:54.532268+00:00',
 'updated': '2022-01-24T02:48:54.532275+00:00',
 'links': {'json': 'https://inspirehep.net/api/authors/2015545?format=json'}}

In [89]:
author_web.json().get('metadata')

{'ids': [{'value': 'M.C.Ponce.1', 'schema': 'INSPIRE BAI'}],
 'name': {'value': 'Ponce, Meritxell Cabrejo'},
 'stub': True,
 '$schema': 'https://inspirehep.net/schemas/records/authors.json',
 'control_number': 2015544}

In [136]:
author_web.json().get('metadata').get('name')['value']

'Muniz, André Luiz Marques'